# Emotion Recognition Using Convoluted Neural Network

By: Aurelius Justin Lim

This notebook aims to create a CNN in order to recognition human emotions. The emotions to be experimented are anger, happiness, and sadness. To do this, the network is trained from a dataset taken from kaggle. A training, test and validation set is used to ensure the correctness and optimal performance of the model. Ultimately, the model created in the notebook is integrated into an application that allows users to upload a photo and allow the model to determine the illustrated emotion. 

**Human Face Emotions By SANIDHYAK** \
**Natural Human Face Images for Emotion Recognition By Sudarshan** \
Reference: \
https://www.kaggle.com/datasets/sanidhyak/human-face-emotions?resource=download \
https://www.kaggle.com/datasets/sudarshanvaidya/random-images-for-face-emotion-recognition?resource=download \

In [5]:

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
from preprocess import preprocess

command = preprocess()
command.run()



Data successfully split into train, validation, and test sets.
Train set: 2644 images - {'sad': 660, 'angry': 780, 'happy': 1204}
Validation set: 331 images - {'sad': 82, 'angry': 98, 'happy': 151}
Test set: 332 images - {'sad': 83, 'angry': 98, 'happy': 151}


In [7]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 2643 images belonging to 3 classes.
Found 331 images belonging to 3 classes.
Found 332 images belonging to 3 classes.


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/50
82/82 [==============================] - 101s 1s/step - loss: 1.0861 - accuracy: 0.4444 - val_loss: 1.1298 - val_accuracy: 0.4563
Epoch 2/50
82/82 [==============================] - 57s 696ms/step - loss: 1.0646 - accuracy: 0.4577 - val_loss: 1.0670 - val_accuracy: 0.4563
Epoch 3/50
82/82 [==============================] - 52s 637ms/step - loss: 1.0615 - accuracy: 0.4669 - val_loss: 1.0360 - val_accuracy: 0.4906
Epoch 4/50
82/82 [==============================] - 64s 777ms/step - loss: 1.0526 - accuracy: 0.4680 - val_loss: 1.0699 - val_accuracy: 0.4531
Epoch 5/50
82/82 [==============================] - 54s 660ms/step - loss: 1.0593 - accuracy: 0.4692 - val_loss: 1.0501 - val_accuracy: 0.4625
Epoch 6/50
82/82 [==============================] - 50s 609ms/step - loss: 1.0448 - accuracy: 0.4776 - val_loss: 1.0573 - val_accuracy: 0.4469
Epoch 7/50
82/82 [==============================] - 52s 633ms/step - loss: 1.0449 - accuracy: 0.4772 - val_loss: 1.0535 - val_accuracy: 0.4500
E

In [10]:
print('Test Generator Samples:', test_generator.samples)
print('Test Generator Batch Size:', test_generator.batch_size)
if test_generator.samples > 0 and test_generator.batch_size > 0:
    steps = test_generator.samples // test_generator.batch_size
    steps = max(steps, 1)  # Ensure at least one step
else:
    raise ValueError("Test generator has invalid samples or batch size.")



test_loss, test_acc = model.evaluate(test_generator, steps=steps)
print('Final Test Loss:', test_loss)
print('Final Test Accuracy:', test_acc)

print('Validation Generator Samples:', validation_generator.samples)
print('Validation Generator Batch Size:', validation_generator.batch_size)
if validation_generator.samples > 0 and validation_generator.batch_size > 0:
    steps = validation_generator.samples // validation_generator.batch_size
    steps = max(steps, 1)  # Ensure at least one step
else:
    raise ValueError("Test generator has invalid samples or batch size.")



val_loss, val_acc = model.evaluate(validation_generator, steps=steps)
print('Final Validation Loss:', val_loss)
print('Final Validation Accuracy:', val_acc)


Test Generator Samples: 332
Test Generator Batch Size: 32
10/10 [==============================] - 4s 477ms/step - loss: 0.8959 - accuracy: 0.5750
Final Test Loss: 0.895933985710144
Final Test Accuracy: 0.574999988079071
Validation Generator Samples: 331
Validation Generator Batch Size: 32
10/10 [==============================] - 3s 344ms/step - loss: 0.9062 - accuracy: 0.5969
Final Validation Loss: 0.9062243700027466
Final Validation Accuracy: 0.596875011920929


In [11]:
# Define your CNN model function
def create_model(hp):
    model = tf.keras.models.Sequential()
    
    # First Conv2D layer
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_1_kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(150, 150, 3)
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    # Second Conv2D layer
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('conv_2_kernel_size', values=[3, 5]),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    # Third Conv2D layer to match the original model structure
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=128),
        kernel_size=hp.Choice('conv_3_kernel_size', values=[3, 5]),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    # Fully connected layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=512, step=64),  # Adjusted range for more flexibility
        activation='relu'
    ))
    model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    # Compile the model
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        metrics=['accuracy']
    )

    return model

# Create a Hyperband Tuner
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',  # Ensure this matches the metric used
    max_epochs=10,
    factor=3,
    directory='logs',
    project_name='cnn_model',
    overwrite=True
)

# Perform the hyperparameter search
tuner.search(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

# Print the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
for key, value in best_hyperparameters.values.items():
    print(f"{key}: {value}")

Trial 1 Complete [00h 07m 23s]
val_accuracy: 0.46525681018829346

Best val_accuracy So Far: 0.46525681018829346
Total elapsed time: 00h 07m 23s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
96                |128               |conv_1_filters
5                 |3                 |conv_1_kernel_size
256               |256               |conv_2_filters
3                 |3                 |conv_2_kernel_size
128               |128               |conv_3_filters
5                 |5                 |conv_3_kernel_size
512               |256               |dense_units
0.2               |0.2               |dropout_rate
0.00010028        |0.0008941         |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
78/83 [===========================>..] - ETA: 10s - loss: 1.0687 - accu

KeyboardInterrupt: 

In [ ]:
model = Sequential([
    # First Conv2D layer with tuned filters and kernel size
    Conv2D(128, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    # Second Conv2D layer with tuned filters and kernel size
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Third Conv2D layer with tuned filters and kernel size
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Flatten layer
    Flatten(),
    
    # Dense layer with tuned units
    Dense(128, activation='relu'),
    
    # Dropout layer with tuned dropout rate
    Dropout(0.1),
    
    # Output layer
    Dense(3, activation='softmax')
])

# Compile the model with tuned learning rate
model.compile(
    optimizer=Adam(learning_rate=0.001325),  # Best learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fit the model using the tuned epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,  # Best number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/50
82/82 [==============================] - 162s 2s/step - loss: 1.1996 - accuracy: 0.4397 - val_loss: 1.0694 - val_accuracy: 0.4688
Epoch 2/50
82/82 [==============================] - 160s 2s/step - loss: 1.0581 - accuracy: 0.4650 - val_loss: 1.0773 - val_accuracy: 0.4625
Epoch 3/50
82/82 [==============================] - 162s 2s/step - loss: 1.0578 - accuracy: 0.4688 - val_loss: 1.0444 - val_accuracy: 0.4719
Epoch 4/50
82/82 [==============================] - 160s 2s/step - loss: 1.0546 - accuracy: 0.4734 - val_loss: 1.0488 - val_accuracy: 0.4719
Epoch 5/50
82/82 [==============================] - 170s 2s/step - loss: 1.0567 - accuracy: 0.4646 - val_loss: 1.0445 - val_accuracy: 0.4688
Epoch 6/50
82/82 [==============================] - 187s 2s/step - loss: 1.0413 - accuracy: 0.4753 - val_loss: 1.0370 - val_accuracy: 0.4812
Epoch 7/50
82/82 [==============================] - 192s 2s/step - loss: 1.0439 - accuracy: 0.4718 - val_loss: 1.0559 - val_accuracy: 0.4719
Epoch 8/50
82

In [ ]:
model.save('ml-model')
